In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

folder_path = "./pdfs/instruction-tune-llama2-extended-guide.pdf"

In [16]:
pdf_docs = PyPDFLoader(folder_path).load_and_split()
pdf_docs[0]

Document(page_content="07/04/2024, 12:41 Extended Guide: Instruction-tune Llama 2\nhttps://www.philschmid.de/instruction-tune-llama-2 1/17philschmidBlogNewsletterTagsProjectsAbout MeContact\nExtended Guide: Instruction-tune Llama\n2\n#GENERATIVEAI#HUGGINGFACE#LLM#LLAMA\nJuly 26, 2023\n13 min read\nView CodeThis blog post is an extended guide on instruction-tuning Llama 2 from\nMeta AI. The idea of the blog post is to focus on creating the\ninstruction dataset, which we can then use to fine-tune the base model\nof Llama 2 to follow our instructions.\nThe goal is to create a model which can create instructions based on\ninput. The idea behind this is that this can then be used for others to\ncreate instruction data from inputs. That's especially helpful if you\nwant to personalize models for, e.g., tweeting, email writing, etc,\nwhich means that you would be able to generate an instruction dataset\nfrom your emails to then train a model to mimic your email writing.\nOkay, so can we get s

In [17]:
len(pdf_docs)

17

Embedding and Vector Database

In [18]:
from langchain.embeddings import OllamaEmbeddings

In [19]:
embedding = OllamaEmbeddings(model="mistral")

In [20]:
from langchain.vectorstores import Chroma

In [22]:
vector_db = Chroma.from_documents(pdf_docs, embedding=embedding, persist_directory=".")

In [24]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama

In [27]:
llm = ChatOllama(model="mistral", chat_format=True)

In [28]:
llm.invoke("HI")

AIMessage(content=" Hello! How can I help you today? If you have any questions or need assistance with something, please don't hesitate to ask. I'm here to help!\n\nHere are a few things I can do:\n- Answer general knowledge questions\n- Help you find information on a specific topic\n- Provide explanations on complex concepts\n- Assist with math problems and equations\n- Generate creative writing ideas or give feedback on your own writing\n- And much more! Let me know what you need.\n\nIf you just want to chat, that's cool too! I don't get tired or bored, so we can talk about anything you'd like for as long as you'd like. Just let me know what's on your mind. :)", response_metadata={'model': 'mistral', 'created_at': '2024-04-07T11:26:39.734102Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 7862984584, 'load_duration': 3256444792, 'prompt_eval_count': 11, 'prompt_eval_duration': 130104000, 'eval_count': 163, 'eval_duration': 4473791000}, id='run-72b0

In [39]:
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_db.as_retriever(), return_source_documents=True)

In [40]:
output = qa.invoke("What is an instruction in the context of LLMs? Cite a passage from the uploaded document related to instructions.", return_sources=True)
output["result"]

' An instruction is a piece of text or prompt provided to an LLM (such as Llama, GPT-4, or Claude) to guide it to generate a response. Instructions allow humans to steer the conversation and constrain the language model\'s output to be more natural, useful, and aligned with the user\'s goals. Here is a passage from the uploaded document explaining instructions:\n\n"An instruction is a piece of text or prompt that is provided to an LLM, like Llama, GPT-4, or Claude, to guide it to generate a response. Instructions allow humans to steer the conversation and constrain the language model\'s output to be more natural, useful, and aligned with the user\'s goals. Crafting clear, well-formulated instructions is key to productive conversations." (07/04/2024, 12:41 Extended Guide: Instruction-tune Llama 2)'

Now, let's do a simple semi-manual benchmark for the quality of this rag setup.

In [55]:
import time
import numpy as np

queries = [
"What is the definition of an instruction for LLM models?",
"How can instructions guide and constrain the output of LLM models?",
"What are some examples of instructions for different capabilities of LLMs, such as Brainstorming,",
"What is the goal of fine-tuning a model to generate instructions based on input?",
"How can synthetic instruction datasets be created for personalizing LLMs and agents?",
"How to define a use case and create a prompt template for instructions?",
"What are the steps to create an instruction dataset, including defining the input and output formats?",
"What research suggests about creating high-quality instruction datasets?",
"What are some methods to create an instruction dataset, such as using existing datasets or synthetically generating new ones?",
"How to use existing LLMs to create synthetic instruction datasets?"
]

outputs = []
latencies = []
for query in queries:
    start = time.time()
    output = qa.invoke(query)
    outputs.append(output)
    end = time.time()
    latencies.append(end - start)

mean_latency = np.mean(latencies)
print(f"Mean latency in seconds: {mean_latency}")

Mean latency in seconds: 11.501888925378973


In [58]:
i=0

In [69]:
print(outputs[i]["query"])
print(outputs[i]["result"])
print(outputs[i]["source_documents"])
i+=1

How to use existing LLMs to create synthetic instruction datasets?
 To create synthetic instruction datasets using existing LLMs, you can follow these general steps:

1. Define the use case and create a prompt template for instructions.
2. Create an instruction dataset by providing inputs and corresponding desired outputs in the form of instructions. For instance, you can use the email request example provided in the guide as a template and modify it to generate different types of instructions such as writing a letter of recommendation or composing a business proposal.
3. Instantiate the LLM and fine-tune it using the instruction dataset, creating an adapter for the model with the help of libraries like Triggers (trl) or the SFTTrainer.
4. Test the model's performance on generating relevant instructions based on new inputs.
5. Merge the adapter weights into the base model and save the merged model for further use.
6. Optionally, push the merged model to a shared hub for public access.
